This kernel was written as part of my tensorflow learning.
If you have any questions or advices, please post below.

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import load_img, ImageDataGenerator
from keras.utils import to_categorical 
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
import matplotlib.pyplot as plt
import os
import random

print(os.listdir('../input/chest-xray-pneumonia/chest_xray'))

In [ ]:
DIR_NORMAL = '../input/chest-xray-pneumonia/chest_xray/train/NORMAL'
DIR_PNEUMONIA = '../input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA'

filenames_N = os.listdir(DIR_NORMAL)
filenames_P = os.listdir(DIR_PNEUMONIA)

filename_N = random.choice(filenames_N)
image_N = load_img(DIR_NORMAL +'/' + filename_N)

filename_P = random.choice(filenames_P)
image_P = load_img(DIR_PNEUMONIA + '/' + filename_P)

f, ax = plt.subplots(1,2, figsize=(30, 30))

# check type
print(type(image_N))

print('Normal image size of {} : {}'.format(filename_N, image_N.size))
ax[0].imshow(image_N)

print('Pneumonia image size of {} : {}'.format(filename_P, image_P.size))
ax[1].imshow(image_P)

In [ ]:
IMAGE_WIDTH = 300
IMAGE_HEIGHT = 300
IMAGE_SIZE = (IMAGE_WIDTH, IMAGE_HEIGHT)

In [ ]:
def create_model():
    
    model = Sequential()
    
    model.add(Conv2D(32, (3,3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(120, (3,3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(200, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

    model.summary()
    
    return model

model = create_model()

In [ ]:
#call backs
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

earlystopping = EarlyStopping(patience=10)
rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                    patience=2, 
                                    verbose=1, 
                                    factor=0.7, 
                                    min_lr=0.00001)

callbacks = [earlystopping, rate_reduction]

In [ ]:
# create train data generator
DIR_TRAIN = '../input/chest-xray-pneumonia/chest_xray/train/'
DIR_VAL = '../input/chest-xray-pneumonia/chest_xray/val/'
DIR_TEST = '../input/chest-xray-pneumonia/chest_xray/test/'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    DIR_TRAIN,
    target_size=IMAGE_SIZE,
    color_mode='grayscale',
    class_mode='binary',
    batch_size=32,
    shuffle=True
)

In [ ]:
# create val generator
val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory(
    DIR_VAL,
    target_size=IMAGE_SIZE,
    color_mode='grayscale',
    class_mode='binary',
    batch_size=16,
    shuffle=False
)

In [ ]:
epochs=50 

history = model.fit_generator(
    train_generator, 
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=1,
    steps_per_epoch=len(filenames_N + filenames_P)//32,
    callbacks=callbacks
)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))
ax1.plot(history.history['loss'], color='b', label="Training loss")
ax1.plot(history.history['val_loss'], color='r', label="validation loss")
ax1.set_xticks(np.arange(1, 18, 1))
ax1.set_yticks(np.arange(0, 20, 1))

ax2.plot(history.history['accuracy'], color='b', label="Training accuracy")
ax2.plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
ax2.set_xticks(np.arange(1, 18, 1))

legend = plt.legend(loc='best', shadow=True)
plt.tight_layout()
plt.show()

In [ ]:
test_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_gen.flow_from_directory(
    DIR_TEST,
    target_size=IMAGE_SIZE,
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=24,
    shuffle=False
)

predict = model.evaluate(test_generator, 
                         steps=np.ceil(624/24),
                         verbose=1,
                         )

In [ ]:
model.save_weights("model.h5")

In [ ]:
# tranfer-learning and try using specific some layers 

from keras.layers import SeparableConv2D, DepthwiseConv2D
from keras.applications.vgg16 import VGG16,  preprocess_input

In [ ]:
def create_new_model():
    
    model_new = Sequential()
    
    model_new.add(Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 1)))
    model_new.add(BatchNormalization())
    model_new.add(MaxPooling2D(pool_size=(2, 2)))
                  
    model_new.add(Conv2D(64, (3,3), activation='relu', padding='same'))
    model_new.add(BatchNormalization())
    model_new.add(MaxPooling2D(pool_size=(2,2)))
    
    model_new.add(SeparableConv2D(128, (3,3), activation='relu', padding='same'))
    model_new.add(BatchNormalization())
    model_new.add(SeparableConv2D(128, (3,3), activation='relu', padding='same'))
    model_new.add(BatchNormalization())
    model_new.add(MaxPooling2D(pool_size=(2,2)))
    
    model_new.add(DepthwiseConv2D((1,1), activation='relu'))
    
    model_new.add(Flatten())
    
    model_new.add(Dense(512, activation='relu'))
    model_new.add(BatchNormalization())
    model_new.add(Dropout(0.5))
    model_new.add(Dense(200, activation='relu'))
    model_new.add(BatchNormalization())
    model_new.add(Dropout(0.5))
    model_new.add(Dense(2, activation='softmax'))
    
    model_new.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model_new.summary()
    
    return model_new

model = create_new_model()

In [ ]:
import h5py

f = h5py.File('../input/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', 'r')

w,b = f['block1_conv1']['block1_conv1_W_1:0'], f['block1_conv1']['block1_conv1_b_1:0']
model.layers[1].set_weights = [w,b]

w,b = f['block1_conv2']['block1_conv2_W_1:0'], f['block1_conv2']['block1_conv2_b_1:0']
model.layers[2].set_weights = [w,b]

w,b = f['block2_conv1']['block2_conv1_W_1:0'], f['block2_conv1']['block2_conv1_b_1:0']
model.layers[4].set_weights = [w,b]

w,b = f['block2_conv2']['block2_conv2_W_1:0'], f['block2_conv2']['block2_conv2_b_1:0']
model.layers[5].set_weights = [w,b]

f.close()
model.summary()    

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

earlystopping = EarlyStopping(patience=6)
rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                    patience=2, 
                                    verbose=1, 
                                    factor=0.7, 
                                    min_lr=0.00001)

callbacks = [earlystopping, rate_reduction]

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    brightness_range=(0, 0.5)
)

train_generator = train_datagen.flow_from_directory(
    DIR_TRAIN,
    target_size=IMAGE_SIZE,
    color_mode='grayscale',
    class_mode='binary',
    batch_size=32,
    shuffle=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory(
    DIR_VAL,
    target_size=IMAGE_SIZE,
    color_mode='grayscale',
    class_mode='binary',
    batch_size=16,
    shuffle=False
)

epochs=50 

history = model.fit_generator(
    train_generator, 
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=1,
    steps_per_epoch=len(filenames_N + filenames_P)//32,
    callbacks=callbacks,
    class_weight={0:0.4, 1:1.0}
)